In [ ]:
import requests
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_input(day, year, save_dir="inputs"):
    """
    Fetches input for a specific Advent of Code day and saves it locally.

    :param day: The day of the Advent of Code (1-25).
    :param year: The Advent of Code year.
    :param save_dir: Directory to save the input file.
    :return: Path to the input file.
    """
    session_token = os.getenv("SESSION_TOKEN")
    if not session_token:
        raise ValueError("Session token not found in environment variables.")

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    headers = {
        "Cookie": f"session={session_token}",
        "User-Agent": "https://github.com/sai-pendyala/Advent-of-Code-2024",
    }

    # Create the directory if it doesn't exist
    Path(save_dir).mkdir(exist_ok=True)
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    # Fetch input
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(input_file, "w") as f:
            f.write(response.text.strip())
        print(f"Input for Day {day} saved to {input_file}")
        return input_file
    elif response.status_code == 404:
        print("Puzzle input is not available yet. Check after the problem is released.")
    else:
        print(f"Failed to fetch input: {response.status_code}")
    return None

In [ ]:
def read_input(day, save_dir="inputs"):
    """
    Reads the content of the input file for a specific day of Advent of Code.

    :param day: The day of the Advent of Code (1-25).
    :param save_dir: Directory where the input file is saved (default is 'inputs').
    :return: Content of the input file as a string.
    """
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    if input_file.exists():
        with open(input_file, "r") as file:
            content = file.read().strip()  # Read and strip any surrounding whitespace
        print(f"Content of Day {day} input:\n{content}")
        return content
    else:
        print(f"Input file for Day {day} does not exist. Please fetch the input first.")
        return None

In [ ]:
# Daily Workflow
from pathlib import Path
from datetime import datetime

# Dynamic Day and Year
now = datetime.utcnow()
DAY = now.day
YEAR = now.year

# Fetch input
fetch_input(DAY, YEAR)

# Read input
input_data = read_input(DAY)

# Day 1

In [ ]:
lefts = []
rights = []
for line in input_data.split("\n"):
  left, right = line.strip().split()
  lefts.append(int(left))
  rights.append(int(right))

In [ ]:
def get_total_distance(lefts, rights):
  total_distance = 0
  for left, right in zip(lefts, rights):
    total_distance += abs(left - right)
  return total_distance

get_total_distance(sorted(lefts), sorted(rights))

In [ ]:
def get_similarity_score(lefts, rights):
  from collections import Counter
  similarity = 0
  right_counts = Counter(rights)
  for left in lefts:
    if left in right_counts:
      similarity += left * right_counts[left]
  return similarity

get_similarity_score(lefts, rights)

# Day 2

In [ ]:
def is_safe(report):
    sign = 1
    if report[1] < report[0]:
        sign = -1

    for i in range(1, len(report)):
        diff = report[i] - report[i - 1]
        if sign * diff < 0 or not 1 <= abs(diff) <= 3:
            return False  
    return True

In [ ]:
total_safe = 0
for line in input_data.split("\n"):
    check = is_safe(list(map(int, line.split())))
    total_safe += check

total_safe

In [ ]:
total_safe2 = 0
for line in input_data.split("\n"):
    report = list(map(int, line.split()))
    for i in range(len(report)):
        first_half = report[:i] if i > 0 else []
        second_half = report[i + 1:] if i < len(report) - 1 else []
        check = is_safe(first_half + second_half)
        if check:
            total_safe2 += 1
            break

total_safe2

# Day 3

In [ ]:
import re
pattern = r"mul\(\d+,\d+\)"
matches = re.findall(pattern, input_data)

total = 0
for match in matches:
    num1, num2 = map(int, match.strip("mul()").split(","))
    total += num1 * num2
total

In [ ]:
import re
pattern = r"mul\(\d+,\d+\)|do\(\)|don't\(\)"
matches = re.findall(pattern, input_data)

total = 0
do = 1
for match in matches:
    if match == "do()":
        do = 1
    elif match == "don't()":
        do = 0
    else:
        num1, num2 = map(int, match.strip("mul()").split(","))
        total += num1 * num2 * do
total

# Day 4

In [ ]:
grid = []
for line in input_data.split("\n"):
    grid.append(list(line))
m, n = len(grid), len(grid[0])

In [ ]:
def dfs(curr, x, y, dx, dy):
    if curr == "XMAS":
        return 1
    if not (0 <= x < m and 0 <= y < n):
        return 0
    if len(curr) > 4:
        return 0
    curr += grid[x][y]
    return dfs(curr, x + dx, y + dy, dx, dy)

In [ ]:
result = 0
for i in range(m):
    for j in range(n):
        if grid[i][j] == "X":
            for di, dj in [(-1, 0), (0, 1), (1, 0), (0, -1), (-1, 1), (1, 1), (1, -1), (-1, -1)]:
                result += dfs("", i, j, di, dj)

result

In [ ]:
result = 0
patterns = ["MAS", "SAM"]
for i in range(1, m - 1):
    for j in range(1, n - 1):
        if grid[i][j] == "A":
            diagonal1 = grid[i - 1][j - 1] + grid[i][j] + grid[i + 1][j + 1]
            diagonal2 = grid[i + 1][j - 1] + grid[i][j] + grid[i - 1][j + 1]
            # print(i, j, diagonal1, diagonal2)
            if diagonal1 in patterns and diagonal2 in patterns:
                result += 1
result

# Day 5

In [ ]:
def get_right_order(graph, degree):
    from collections import deque
    q = deque()
    for u, d in degree.items():
        if d == 0:
            q.append(u)
    right_order = []
    while q:
        node = q.popleft()
        right_order.append(node)
        for v in graph[node]:
            degree[v] -= 1
            if degree[v] == 0:
                q.append(v)
                
    return right_order

In [ ]:
from collections import defaultdict

rules = defaultdict(set)
updates = []

for line in input_data.split("\n"):
    if line == '':
        continue 
    if '|' in line:
        u, v = line.split('|')
        rules[u].add(v)
    else:
        updates.append(line.split(','))

print(rules, updates)

In [ ]:
result1 = 0
result2 = 0
for update in updates:
    degree = {u: 0 for u in update} 
    graph = defaultdict(set)

    for u in update:
        graph[u] = rules[u] & set(update)
        for v in graph[u]:
            degree[v] += 1
    right_order = get_right_order(graph, degree)
    if right_order == update:
        result1 += int(right_order[(len(update) - 1) // 2])
    else:
        result2 += int(right_order[(len(update) - 1) // 2])
    
result1, result2

# Day 6

In [ ]:
grid = []
for line in input_data.split("\n"):
    grid.append(list(line))

m, n = len(grid), len(grid[0])
start = None
directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
for i in range(m):
    for j in range(n):
        if grid[i][j] == '^':
            start = (i, j)
            break
    if start is not None:
        break

In [ ]:
def travel(x, y, d_idx, path):
    while True:
        path.add((x, y))
        dx, dy = directions[d_idx]
        nx, ny = x + dx, y + dy 
        if not (0 <= nx < m and 0 <= ny < n):
            break 
        if grid[nx][ny] == '#':
            d_idx = (d_idx + 1) % 4
        else:
            x, y = nx, ny

In [ ]:
path = set()
travel(start[0], start[1], 0, path)
print(len(path))

In [ ]:
def simulate(x, y):
    vis = set()
    d_idx = 0
    while True:
        if (x, y, d_idx) in vis:
            return True
        vis.add((x, y, d_idx))

        dx, dy = directions[d_idx]
        nx, ny = x + dx, y + dy 

        if not (0 <= nx < m and 0 <= ny < n):
            break 

        if grid[nx][ny] == '#':
            d_idx = (d_idx + 1) % 4
        else:
            x, y = nx, ny
    
    return False

x, y = start
count = 0
for i, j in path:
    grid[i][j] = '#'
    count += simulate(x, y)
    grid[i][j] = '.'

count